<a href="https://colab.research.google.com/github/Patcharaporn2093/Data-ANA-ML/blob/main/Predictive_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this challenge, you will be provided with a carefully curated dataset that spans a real-world problem. Your mission is to develop and fine-tune machine learning algorithms to predict a target variable based on the given features.

The dataset for this competition has been anonymously collected. The input data is composed on categorial (Major) and numeric (Year and Score?). Changes to feature selection process are welcomed.

Data contains information about students' performance in one semester. Major is a number code that indicates the field of study the students are enrolling. Year is a number of year in school. Score1 to Score16 are scores students get for each assignment. The maximum score for each assignment is 15. A missing value means that students do not turn in that assignment. Final is the final examination score. This field is what we want to predict. Train data will provide all information while test data will not provide Final column.

Your objective is to write a python code for predicting Final score from other features.

In [1]:
import requests

url = 'https://drive.google.com/uc?id=1PwJSq3x_v-YyDrYssWDWLETM5xc4n0iS'
output = 'train.csv'

response = requests.get(url)
with open(output, 'wb') as f:
    f.write(response.content)

In [3]:
import requests

url = 'https://drive.google.com/uc?id=1J4QZDzTmQ97Ijr4ZUKlCZoRhpYKYCYjt'
output = 'test.csv'

response = requests.get(url)
with open(output, 'wb') as f:
    f.write(response.content)


In [4]:
# 1. Import
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# 2. โหลดข้อมูล
data_train = pd.read_csv('train.csv')  # โหลดข้อมูลการฝึก (training set)
data_test = pd.read_csv('test.csv')    # โหลดข้อมูลทดสอบ (test set)

# 3. แทนที่ค่าที่ขาดหายด้วยเลข 0
data_train = data_train.fillna(0)
data_test = data_test.fillna(0)

# 4. แบ่งข้อมูลเป็น Features และ Target Variable ในชุดข้อมูลการฝึก
X = data_train.drop(['ID', 'Final'], axis=1)
y = data_train['Final']

# 5. ดำเนินการก่อนประมวลผลข้อมูล - จัดการกับค่าที่ขาดหายและปรับขนาดคุณลักษณะ
numeric_cols = X.select_dtypes(include=['number']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# 6. แทนที่ค่าที่ขาดหายด้วย 0 สำหรับคอลัมน์ที่เป็นตัวเลข
numeric_transformer = SimpleImputer(strategy='constant', fill_value=0)

# 7. ใช้ SimpleImputer เพื่อจัดการค่าที่ขาดหายและทำการ one-hot encode สำหรับคอลัมน์ที่เป็นข้อความ
categorical_transformer = SimpleImputer(strategy='constant', fill_value='missing')

# 8. รวม transformers เพื่อสร้าง preprocessor โดยใช้ ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# 9. ใช้ preprocessor เพื่อ fit และ transform ข้อมูลการฝึก
X_preprocessed = preprocessor.fit_transform(X)

# 10. ใช้ RandomForestRegressor เป็นโมเดล
model = LinearRegression()

# 11. Fit โมเดลกับข้อมูลการฝึกที่ผ่านการประมวลผลแล้ว
model.fit(X_preprocessed, y)

# 12. ทำการประมวลผลข้อมูลทดสอบ
X_test = data_test.drop('ID', axis=1)
X_test_preprocessed = preprocessor.transform(X_test)

# 13. ทำการทำนายผลลัพธ์บนข้อมูลทดสอบ
test_preds = model.predict(X_test_preprocessed)

# 14. สร้าง DataFrame สำหรับไฟล์ submission
submission_data_linear = pd.DataFrame({'ID': data_test['ID'], 'Final': test_preds})

# 15. บันทึกไฟล์ submission
submission_data_linear.to_csv('submission_linear.csv', index=False)